In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from pprint import pprint
import logging
import pathlib

In [2]:
# necessary paths
NOTEBOOK_PATH = pathlib.Path().resolve()
LOG_DIRECTORY = NOTEBOOK_PATH.parent / "log"

We will <b>build a logistic regression model from scratch</b> and use it to <b>classify the iris dataset</b>.

# Load dataset
- Load the iris dataset from sklearn, use data for the classes 0 and 1 only for this example
- Split chosen data into train set and test set

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
## testing 
# X = np.append(iris.data[:3], iris.data[50:53], axis=0)
# y = np.append(iris.target[:3], iris.target[50:53], axis=0)[:, np.newaxis]
X = iris.data[:100]
y = iris.target[:100][:, np.newaxis]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify=y, random_state=42)

# Construct a logistic regression model

## Compute prediction

### Write the sigmoid function
- Recall a sigmoid function is defined by:<br> 
$$S(x) = \frac{1}{1 + e^{-x}}$$

In [4]:
# sigmoid function
def sigmoid(x):
    """
    Args:
        x (numpy array): the linear combination of features
    
    Returns:
        return a probability (0-1)
    """
    return 1 / (1 + np.exp(-x))

In [5]:
# test sigmoid
sigmoid(0)

0.5

### Write the predict function
- The predicted probabiliy of a logistic regression model is given by:<br>
$$h_{\beta}(x) = S(\beta \cdot x)$$

In [6]:
# y_hat function
def y_hat(beta, X):
    """
    Args:
        beta (numpy array): weight vector in linear regression, dimension 1 x n
        X (numpy array): training data, dimension m x n
        
    Returns:
        return predicted probabilities, dimension 1 x n
    """
    return sigmoid(np.dot(X, beta))

In [7]:
# choose the first two rows of training data for testing
beta = np.zeros((X.shape[1], 1))
Xtry = Xtrain[0:2]
ytry = ytrain[0:2]

In [8]:
# test yhat with trial data
yhat = y_hat(beta, Xtry)
yhat

array([[0.5],
       [0.5]])

## Write the cost function
- Suppose we have $m$ rows of training data and $n$ features
- Recall the cost function of logistic regression is given by:<br> 
$$J(\beta) = -\frac{1}{m}\sum_{i=1}^{m}[y_i log(\hat{y_i})+(1-y_i)log(1-\hat{y_i})]$$
where $y_i$ and $\hat{y_i}$ are the true values and predicted values of i-th row of training data respectively.

In [9]:
# cost function per row of data
def cost_per_row(y, yhat):
    if y == 1:
        return np.multiply(y, np.log(yhat)) 
    else:
        return np.multiply(1-y, np.log(1 - yhat))

In [10]:
# test cost_per_row
cost_per_row(0, 0.5)

-0.6931471805599453

In [11]:
# total cost
def cost(Y, Yhat):
    """
    Args:
        Y (numpy array): labels of training data, dimension 1 x m
        Yhat (numpy array): predictied values of training data, dimension 1 x m
        
    Returns:
        return the cost of beta
    """
    m = len(y)
    df1 = pd.DataFrame(data={'y': Y.ravel(), 'yhat': Yhat.ravel()})
    diff = df1.apply(lambda row: cost_per_row(row['y'], row['yhat']), axis=1).to_numpy().reshape(-1, 1)
    return -np.mean(diff)

In [12]:
# test cost
cost(ytry, yhat)

0.6931471805599453

## Compute the gradient
- We will use gradient descent for optimizing the cost function above.
- Write a function to compute gradient for updating the weight $\beta$ when feeding each row of training data
- Recall in sec 2.1 the weight updating formula is given by:<br>
$$\beta_{k+1} = \beta_{k} - \alpha \frac{\partial J}{\partial \beta}(\beta_k)$$
where $\alpha$ is a fixed learning rate, $k$ is the number of iterations/epochs.
- The gradient is<br><br>
$$\frac{\partial J}{\partial \beta} = (\frac{\partial J}{\partial \beta_0}, \frac{\partial J}{\partial \beta_1},..., \frac{\partial J}{\partial \beta_n})$$
- We only need to compute $$\frac{\partial J}{\partial \beta_j} = \frac{1}{m}\sum_{i=1}^{m}(\hat{y_i}-y_i)x_{ij}$$ for each $j = 0, 1, 2,... ,n$.
- Derivation of the gradient formula of logistic regression can be found at p.18 of:<br> 
https://see.stanford.edu/materials/aimlcs229/cs229-notes1.pdf

In [13]:
# gradient function
def gradient(beta, X, y, yhat):
    # initialize the gradient as a zero vector
    grad = np.zeros(beta.shape)
    
    # compute delta J/delta beta for each j
    for j in range(X.shape[1]):
        first = np.multiply(yhat - y, X[:, j][: ,np.newaxis])
        grad[j] = np.mean(first)
    
    return grad

In [14]:
# test gradient
gradient(beta, Xtry, ytry, yhat)

array([[-3.  ],
       [-1.45],
       [-2.05],
       [-0.7 ]])

## Write the gradient descent function
- Initialize beta as a zero vector, for each epoch, feed the whole train set to calculate: 
    1. yhat (prediction of training data using existing beta)
    2. gradient vector
    3. update beta using formula in 2.3
    4. cost/loss of existing prediction
- Remember to add a constant column (e.g. a column with all 1) to Xtrain that represents the <b>intercept</b> of linear regression

In [15]:
%%time
logger = logging.getLogger(__name__)
logging.basicConfig(filename=LOG_DIRECTORY / "train.log", encoding='utf-8', level=logging.DEBUG, force=True)

def train_grad_desecnt(X, y, epochs=1000, alpha=0.1):
    """
    Args:
        epochs (int): no. of iterations allowed for optimization
        alpha (floats): learning rate
    
    Returns:
        beta (numpy array): the final weight
        loss (float): the final cost
    """
    m, n = X.shape
    beta = np.zeros((n+1, 1))    # +1 accounts for constant coefficient beta_0
    X = np.hstack((np.ones((m, 1)), X))    # add a constant column for constant coefficient
    losses = []
    
    # training loop
    for epoch in range(epochs):
        yhat = y_hat(beta, X)
        grad = gradient(beta, X, y, yhat)
                        
        # update beta and loss
        beta -= alpha*grad
        loss = cost(y, yhat)
        losses.append(loss)
        
        # log result of each epoch
        logging.info("*"*50)
        logging.info(f'epoch = {epoch}')
        logging.info(f'beta = {beta}')
        logging.info(f'grad = {grad}')
        #logging.info(f'yhat = {yhat[:50]}')
        #logging.info(f'y = {y[:50]}')
        logging.info(f'loss = {loss}')
        
    return beta, losses

CPU times: user 296 µs, sys: 556 µs, total: 852 µs
Wall time: 705 µs


In [16]:
%%time
# fit training set into train_gard_descent function
beta_final, losses = train_grad_desecnt(Xtrain, ytrain)
print('beta_final = ', beta_final)
print('loss = ', losses[-1])

beta_final =  [[-0.35313805]
 [-0.46177555]
 [-2.09037366]
 [ 3.01293972]
 [ 1.33824509]]
loss =  0.0077567443047278925
CPU times: user 1.92 s, sys: 21.9 ms, total: 1.94 s
Wall time: 1.95 s


## Write the predict function for new data
- The function should use beta calculated in gradient descent and Xtest to return predicitons and predicted probabilities.

In [17]:
# use trained beta to predict new data
def predict(beta, X):
    """
    Args:
        X (numpy array): feature to predict
        
    Returns:
        y (numpy array): predict results (0 or 1)
    """
    m = X.shape[0]
    X = np.hstack((np.ones((m, 1)), X))    # add constant column to X for intercept
    pred_proba = sigmoid(np.dot(X, beta)).ravel()
    print(pred_proba.shape)
    pred = np.array([round(x) for x in pred_proba])
    return pred, pred_proba

In [18]:
# find accuracy of model on test data
from sklearn.metrics import accuracy_score
ypred, ypred_proba = predict(beta_final, Xtest)
accuracy_score(ytest, ypred)

(25,)


1.0

In [19]:
# see if the predicted probabilities look normal 
ypred_proba

array([9.95799570e-01, 9.98890025e-01, 9.97852823e-01, 4.20178672e-03,
       9.97741322e-01, 9.98418422e-01, 5.11400991e-02, 9.98265663e-01,
       9.93678890e-01, 9.96196031e-01, 7.46767781e-03, 9.74623091e-01,
       5.47415895e-04, 6.81204464e-03, 9.95258850e-01, 6.50663366e-03,
       3.03637496e-03, 9.99510463e-01, 1.12998599e-02, 1.20260491e-03,
       4.47302413e-03, 2.15560638e-02, 1.03092611e-02, 4.60555138e-03,
       9.78354426e-01])

The accuracy is 1.0 and the prediced probabilities look normal, great! Note that this implementation is only a simple version, it may not work well in more complex dataset. More details like <b>regularization</b> and <b>weighted cost function</b> (for imbalanced data) can be added to advance the model. Also, data preprocessing like <b>feature scaling</b> may be required.